# Big Assignment of Parallel Processing (Class A)
## Sales Transaction Data Analysis Using Pyspark\

1. siapa dah (nim)

Kaggle link for the dataset: https://www.kaggle.com/datasets/gabrielramos87/an-online-shop-business

In [2]:
import psutil
print(psutil.cpu_percent())

1.4


In [1]:
# %pip install pyspark
# %pip install mpi4py
# print(psutil.cpu_percent())

In [3]:
from mpi4py import MPI
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
print(psutil.cpu_percent())

1.9


In [4]:
spark = SparkSession.builder.appName("Game Sales Analysis").getOrCreate()
print(psutil.cpu_percent())

1.8


In [ ]:
import time
mulai_ukur = time.time()

In [5]:
df = spark.read.csv("game_info.csv", header=True, inferSchema=True)
print(psutil.cpu_percent())

2.0


In [7]:
df.show(10)
print(psutil.cpu_percent())

+------+--------------------+--------------------+----------+----------+-----+-------------------+--------------------+------+----------+--------+------------------+-------------+-----------------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+------------+----------------+------------------+-------------------+-------------------+--------------------+--------------------+
|    id|                slug|                name|metacritic|  released|  tba|            updated|             website|rating|rating_top|playtime|achievements_count|ratings_count|suggestions_count|game_series_count|reviews_count|           platforms|          developers|              genres|          publishers| esrb_rating|added_status_yet|added_status_owned|added_status_beaten|added_status_toplay|added_status_dropped|added_status_playing|
+------+--------------------+--------------------+----------+----------+-----+-------------------+------------

In [5]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Load CSV hanya di rank 0
df = None
if rank == 0:
    df = pd.read_csv("game_info.csv")
    chunks = np.array_split(df, size)
else:
    chunks = None

# Broadcast potongan data
local_chunk = comm.scatter(chunks, root=0)

# FILTER: metacritic > 80
filtered_chunk = local_chunk[local_chunk["metacritic"] > 80]

# REDUCE: jumlah ratings_count
local_sum = filtered_chunk["ratings_count"].sum()
total_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

if rank == 0:
    print("Total ratings_count untuk metacritic > 80:", total_sum)


C:\Users\yoga7.RAKASYA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Total ratings_count untuk metacritic > 80: 427925


In [1]:
from pyspark import SparkContext
import csv

sc = SparkContext()

rdd = sc.textFile("game_info.csv")
header = rdd.first()
data = rdd.filter(lambda x: x != header).map(lambda row: next(csv.reader([row])))

# FILTER: metacritic > 80
filtered = data.filter(lambda x: x[3] and x[3].isdigit() and int(x[3]) > 80)

# REDUCE: total ratings_count
ratings_count_index = 12  # Sesuaikan jika urutan kolom berubah
ratings = filtered.map(lambda x: int(x[ratings_count_index]) if x[ratings_count_index].isdigit() else 0)
total = ratings.reduce(lambda a, b: a + b)

print("Total ratings_count (PySpark):", total)


Total ratings_count (PySpark): 427925


In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("GameProcessing").getOrCreate()

df = spark.read.csv("game_info.csv", header=True, inferSchema=True)
df.createOrReplaceTempView("games")

# Contoh Query: Ambil game dengan metacritic > 80 dan tampilkan 5 teratas
result = spark.sql("""
    SELECT name, rating, ratings_count
    FROM games
    WHERE rating IS NOT NULL
    ORDER BY rating DESC, ratings_count DESC
    LIMIT 20
""")

print("Hasil Query:")
result.show()

Hasil Query:
+--------------------+--------------------+-------------+
|                name|              rating|ratings_count|
+--------------------+--------------------+-------------+
|        "The ""Quiet|http://www.nostat...|            0|
|"Электроника ИМ-0...|http://facebook.c...|            0|
|           Diablo IV|                 5.0|            5|
|The World Ends wi...|                4.86|            7|
|Astro Boy: Omega ...|                4.83|            6|
|         Hide Online|                4.83|            6|
|       Kara no Shojo|                4.83|            6|
|     Persona 5 Royal|                4.81|          118|
|Shovel Knight: Sh...|                4.78|            9|
|The Witcher 3: Ga...|                4.77|          439|
| Mass Effect Trilogy|                4.76|          104|
|The Witcher 3: Wi...|                4.75|          334|
|METAL GEAR SOLID ...|                4.75|           23|
|Cataclysm: Dark D...|                4.75|            7|
|